In [73]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline

from sklearn.feature_selection import SelectKBest,chi2

In [74]:
df = pd.read_csv("ttrain.csv")

In [75]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## Plan
### Imput  Missing Value ---->  

### One Hot Encoding (Sex and Embarkde )   --->  

### Scaling (similar numeric range) --->

### Feature Selection (select 5 top columns) 

### Model train

In [76]:
# we dont find this column helpfull so we are dropping it 
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [77]:
X_train, x_test , y_train, y_test = train_test_split(df.drop(columns=['Survived']),
                                                    df['Survived'],test_size=0.2,random_state=42)

In [78]:
X_train.shape

(712, 7)

In [79]:
y_train.shape

(712,)

## Columns Transformer  :
* pass list of tuple

In [57]:
## while creating pipeline good strategy is to call column by their index rather than their name

In [80]:
# imputation transformr


transformer1 = ColumnTransformer([
('impute_age',SimpleImputer(),[2]),
('impute_embarked',SimpleImputer(strategy= 'most_frequent'),[6]),
    ],remainder = 'passthrough')


In [59]:
# Encoding Transformer : one hot encoding

transformer2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6]) #1 and 6 are index number 
],remainder= 'passthrough')


In [81]:
## Scaling 

from sklearn.preprocessing import MinMaxScaler # feature selection


transformer3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10)) # slice apply transformer in this range 0,10 
    # why 10  we have 7 columns -- sex -- > 2 more column and embarked --> 3 more column  --> 7 -2 + 2+3 = 10
])



In [82]:
## feature scaling 

transformer4 =  SelectKBest(score_func= chi2, k=8 ) #using top 8 columns 

In [83]:
transformer5 = DecisionTreeClassifier()

# Create Pipeline
list of tuple having name and object 

In [85]:
pipe = Pipeline( #this is pipe line object
[
    ('trf1',transformer1),
     ('trf2',transformer2),
     ('trf3',transformer3),
     ('trf4',transformer4),
     ('trf5',transformer5)
])

In [86]:
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),


### make pipeline

In [124]:
pipe = make_pipeline(transformer1,transformer2,transformer3,transformer4,transformer5)
pipe

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001E423229550>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [96]:
  # for training model use  fit and if your are only doing preprocsing then use fit_transform
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001E423229550>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [108]:
pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_ # this is mean value 

array([29.49884615])

In [125]:
pipe.named_steps['columntransformer-1'].transformers_[1][1].statistics_
pipe.named_steps


{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x000001E423229550>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [95]:
from sklearn import set_config
set_config(display='diagram')

In [98]:
# predict 
y_pred = pipe.predict(x_test)
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

#  Accuracy Score 

In [118]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

#  Cross Validation Using Pipeline

In [144]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,scoring='accuracy',cv=5).mean()

0.6391214419383433

## Hyper Parameter Tuning : Grid Search Cv`m

In [145]:
params = {
    'decisiontreeclassifier_max_depth':[1,2,3,4,5,None] #name should be same syntax :)  name_max_depth
}

In [149]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)
 

ValueError: Invalid parameter decisiontreeclassifier_max_depth for estimator Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTran

In [147]:
grid.best_params_ 

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

# Exporting the Pipe Line

In [152]:
import pickle 
pickle.dump(pipe,open('pipe.pkl','wb'))